In [111]:
import Pkg

Pkg.add("JLD2")

    Updating registry at `C:\Users\matsz\.julia\registries\General.toml`
   Resolving package versions...
   Installed FileIO ─ v1.16.3
   Installed JLD2 ─── v0.4.51
    Updating `C:\Users\matsz\.julia\environments\v1.10\Project.toml`
  [033835bb] + JLD2 v0.4.51
    Updating `C:\Users\matsz\.julia\environments\v1.10\Manifest.toml`
  [5789e2e9] + FileIO v1.16.3
  [033835bb] + JLD2 v0.4.51
Precompiling project...
  ✓ FileIO
  ✓ Plots → FileIOExt
  ✓ JLD2
  ✓ TaylorSeries → TaylorSeriesJLD2Ext
  4 dependencies successfully precompiled in 79 seconds. 538 already precompiled.


In [1]:
using Revise
using Turing, MCMCChains
using XLSX
using DataFrames, TimeSeries
using StatsPlots
using LinearAlgebra
using CSV
using StatsBase
include("utils.jl")
include("financial_bvar_temp.jl")
using .FinancialBVAR
using JLD2

In [2]:
data = TimeArray(CSV.File("usa_data_transformed.csv"),timestamp=:timestamp)
start = Date(1993,03,01)
end_d = Date(2023, 12, 31)
s_data = to(from(data, start),end_d)

124×6 TimeArray{Float64, 2, Date, Matrix{Float64}} 1993-03-01 to 2023-12-01
┌────────────┬─────────────────┬───────────────┬─────────────┬───────────────┬──
│            │ Total_Stock_CPI │ Price_10Y_CPI │ T_Bill_CPI  │ Dividend_Yeld │ ⋯
├────────────┼─────────────────┼───────────────┼─────────────┼───────────────┼──
│ 1993-03-01 │       0.0150658 │     0.0543577 │ -0.00250148 │     0.0281965 │ ⋯
│ 1993-06-01 │       0.0136547 │     0.0292271 │  0.00478562 │     0.0279908 │ ⋯
│ 1993-09-01 │       0.0342312 │       0.03506 │ -0.00169417 │     0.0270317 │ ⋯
│ 1993-12-01 │       0.0226641 │    -0.0219636 │  0.00410946 │     0.0266883 │ ⋯
│ 1994-03-01 │      -0.0571526 │    -0.0622961 │ 0.000425924 │     0.0285162 │ ⋯
│ 1994-06-01 │      0.00958316 │    -0.0293652 │  0.00338014 │     0.0285113 │ ⋯
│ 1994-09-01 │        0.026626 │   -0.00884948 │  0.00390608 │     0.0280574 │ ⋯
│ 1994-12-01 │      0.00488791 │  -0.000941161 │  0.00828514 │     0.0283289 │ ⋯
│     ⋮      │        ⋮        │ 

In [3]:
vmodel = VARModel(s_data)
scenarios = simulate(vmodel,60, 100_000)
returns_scenarios = exp.(scenarios[1:3,:,:])


3×100000×61 Array{Float64, 3}:
[:, :, 1] =
 1.1293   1.1293   1.1293   1.1293   …  1.1293   1.1293   1.1293   1.1293
 1.06732  1.06732  1.06732  1.06732     1.06732  1.06732  1.06732  1.06732
 1.0104   1.0104   1.0104   1.0104      1.0104   1.0104   1.0104   1.0104

[:, :, 2] =
 0.979432  0.946263  1.04261  1.01532  …  0.942822  1.07503   0.933708
 0.946301  0.930844  1.02242  1.00843     0.944339  0.924086  0.954406
 0.9937    1.01172   1.01277  1.00762     0.99154   1.00702   0.997066

[:, :, 3] =
 0.926114  1.01314  1.09145   1.06716   …  0.973282  1.06272   1.00492
 1.02073   1.07591  0.958194  1.05715      1.01477   0.978223  0.974937
 1.0019    1.00781  1.00688   0.994819     1.01      1.01135   1.0073

;;; … 

[:, :, 59] =
 0.988462  1.04014  0.896105  0.973972  …  0.956774  0.937516  1.04046
 0.959515  1.08368  1.08266   1.04963      0.922665  1.03037   0.930883
 0.983632  1.01763  1.00491   1.00621      0.990994  0.996649  0.995334

[:, :, 60] =
 0.923048  1.06075  0.965996  0

In [4]:
includet("kmeans_moments_lattice.jl")
using .LatticeGeneration

In [5]:
returns_scenarios

3×100000×61 Array{Float64, 3}:
[:, :, 1] =
 1.1293   1.1293   1.1293   1.1293   …  1.1293   1.1293   1.1293   1.1293
 1.06732  1.06732  1.06732  1.06732     1.06732  1.06732  1.06732  1.06732
 1.0104   1.0104   1.0104   1.0104      1.0104   1.0104   1.0104   1.0104

[:, :, 2] =
 0.979432  0.946263  1.04261  1.01532  …  0.942822  1.07503   0.933708
 0.946301  0.930844  1.02242  1.00843     0.944339  0.924086  0.954406
 0.9937    1.01172   1.01277  1.00762     0.99154   1.00702   0.997066

[:, :, 3] =
 0.926114  1.01314  1.09145   1.06716   …  0.973282  1.06272   1.00492
 1.02073   1.07591  0.958194  1.05715      1.01477   0.978223  0.974937
 1.0019    1.00781  1.00688   0.994819     1.01      1.01135   1.0073

;;; … 

[:, :, 59] =
 0.988462  1.04014  0.896105  0.973972  …  0.956774  0.937516  1.04046
 0.959515  1.08368  1.08266   1.04963      0.922665  1.03037   0.930883
 0.983632  1.01763  1.00491   1.00621      0.990994  0.996649  0.995334

[:, :, 60] =
 0.923048  1.06075  0.965996  0

In [6]:
s = sum(returns_scenarios[:,:,21:25]; dims=3)
s = dropdims(s; dims= 3)
mean(s[1,:,:])

5.077596326109633

In [7]:
returns_scenarios_ann = zeros(3,100_000,15)
for asset in 1:3
    returns_scenarios_ann[asset,:,:] = annualise(returns_scenarios[asset,:,2:end],4)
end
returns_scenarios_ann


3×100000×15 Array{Float64, 3}:
[:, :, 1] =
 3.76473  4.02786  4.17353  4.0695   …  3.9701   4.20055  4.1131   3.94081
 4.07999  3.92169  3.90717  4.05758     4.07167  3.96846  3.87244  3.94703
 4.01658  4.01754  4.01204  4.02804     3.9924   4.00081  4.03415  4.00876

[:, :, 2] =
 4.38357  4.02791  3.93663  4.04055  …  4.10953  4.08391  4.0882   4.0757
 3.9266   4.04431  4.14215  3.87492     3.95003  4.12177  4.0114   3.98334
 4.02792  3.99966  4.0478   4.01454     4.03583  4.03748  4.03188  4.01246

[:, :, 3] =
 4.23838  4.0207   4.05491  4.332    …  3.94288  4.23528  4.19308  4.21327
 4.11607  4.00276  3.94227  4.04337     3.98721  3.91891  3.99011  3.99213
 4.02502  4.01691  4.00331  4.03728     4.02387  4.00779  4.02722  4.01467

;;; … 

[:, :, 13] =
 4.22422  4.11483  4.17099  4.11825  …  4.05568  4.05374  3.802    3.92422
 3.99522  3.97931  4.00476  3.9763      4.04283  4.11551  4.04629  4.17302
 3.97276  3.97947  3.9899   4.01437     3.99229  3.98614  3.9999   3.98611

[:, :, 14

In [8]:
res = generate_lattice(returns_scenarios_ann, 9,15)


Lattice([3.9677712225053408 4.16037020371943 … 4.16372659274145 4.334347452237141; 4.031548776342026 3.94038115465565 … 4.118351094526266 4.152481274854603; … ; 4.193312761680589 4.201737098645205 … 4.099247950548817 4.050641148485665; 3.8618455995215575 4.214718728122945 … 3.9602295733772532 4.020421172044646;;; 3.9680391768668932 4.028895168282186 … 3.9012427516249946 3.98949119069828; 4.146568671739699 3.968563723668759 … 3.931966989284409 3.8988848032794543; … ; 3.8919873544774855 4.0950358500472674 … 4.057916016128649 3.930322287849103; 4.047292513828673 3.8905646653717985 … 3.9355466328621453 4.018739152989814;;; 4.007822383717923 4.0044176257783874 … 4.012268551028478 4.019089567679289; 4.036137207513852 3.9974773488046003 … 4.015299513734185 3.9879095080358415; … ; 3.9672447305672325 4.026089939369318 … 3.9873445864987165 3.99581885190879; 3.9913236237948433 3.9686028243310454 … 3.964024528275138 4.019167791690985], [[1.0;;], [0.11909 0.15429 … 0.12376 0.10997], [0.096901503064

In [9]:
res.states

15×9×3 Array{Float64, 3}:
[:, :, 1] =
 3.96777  4.16037  4.18925  4.09956  …  3.97303  4.33212  4.16373  4.33435
 4.03155  3.94038  3.89248  4.12452     4.33256  4.23327  4.11835  4.15248
 3.87012  4.13348  4.01581  3.98915     4.32199  4.10192  4.22698  3.92185
 4.10107  3.86411  4.17404  4.01842     4.27912  4.0539   3.9208   3.98452
 3.87026  4.12014  4.06562  4.03034     4.26376  4.01635  4.26417  3.87489
 4.1452   3.86483  4.27788  4.02493  …  3.97073  3.89763  4.07431  4.1102
 4.11814  3.87392  4.05988  4.09336     4.25705  4.03031  4.26721  4.02192
 4.09335  4.08223  3.89452  4.04233     4.08721  4.29983  3.98631  3.85772
 4.035    4.04605  3.90013  4.08929     4.29743  4.23313  3.84265  4.13505
 3.8597   4.04135  4.12343  4.07518     4.27278  3.88037  4.03325  4.05705
 4.1379   4.04326  3.92743  3.84007  …  3.96726  4.29944  4.14862  4.03208
 4.05044  4.27992  3.88236  4.03076     4.13137  4.07075  4.25599  4.0472
 4.10476  4.18465  4.19189  4.08859     3.95082  3.86047  3.9568

In [16]:
res.probs

16-element Vector{Matrix{Float64}}:
 [1.0;;]
 [0.11909 0.15429 … 0.12376 0.10997]
 [0.0969015030649089 0.07758837853724074 … 0.12981778486858678 0.06364934083466287; 0.03772117441182189 0.16546762589928057 … 0.1433015749562512 0.14874586817032862; … ; 0.17129928894634777 0.07748868778280543 … 0.10633484162895927 0.030300581771170006; 0.06874602164226608 0.17832136037101026 … 0.12457943075384195 0.09720832954442121]
 [0.06175094167332496 0.14849902979111973 … 0.1168816345166077 0.10170071909599361; 0.10841383865907027 0.07827594748575675 … 0.1189001733960862 0.11171662125340599; … ; 0.10216840772396328 0.0389363722697056 … 0.1494935106046217 0.07589427033871478; 0.1498819423057181 0.07596755979878862 … 0.08233240940355199 0.1498819423057181]
 [0.18257781491539588 0.0931689993733027 … 0.05984959264675162 0.11499895550449134; 0.15787312666803532 0.18066105522479983 … 0.10983370971053172 0.13405871484294807; … ; 0.12358650164722643 0.10337458819339329 … 0.09545009349122963 0.07479298370581

In [17]:
round.(res.states[5,:,:]; digits=4)

9×3 Matrix{Float64}:
 3.8703  4.1223  4.0231
 4.1201  4.0083  3.9887
 4.0656  4.1002  4.0074
 4.0303  3.9704  4.0171
 4.0902  4.1193  4.0398
 4.2638  3.9036  3.9936
 4.0163  3.8989  3.9776
 4.2642  4.0198  4.0205
 3.8749  4.0194  3.9938

In [18]:
round.(res.states[6,:,:]; digits=4)

9×3 Matrix{Float64}:
 4.1452  4.0971  4.0377
 3.8648  4.0412  3.9954
 4.2779  3.9924  4.013
 4.0249  3.9691  4.0154
 4.2272  3.8879  3.9846
 3.9707  3.9163  3.9769
 3.8976  4.1346  4.0282
 4.0743  4.101   4.0072
 4.1102  4.0101  3.9863

In [19]:
round.(res.probs[6]; digits=2)

9×9 Matrix{Float64}:
 0.06  0.17  0.11  0.12  0.03  0.12  0.17  0.07  0.16
 0.08  0.15  0.14  0.13  0.07  0.11  0.1   0.12  0.11
 0.08  0.1   0.11  0.19  0.1   0.11  0.08  0.14  0.09
 0.11  0.07  0.12  0.18  0.18  0.08  0.03  0.16  0.07
 0.13  0.12  0.12  0.14  0.06  0.07  0.12  0.06  0.18
 0.11  0.09  0.11  0.19  0.11  0.07  0.08  0.1   0.13
 0.14  0.09  0.13  0.18  0.14  0.06  0.04  0.12  0.09
 0.06  0.14  0.12  0.14  0.08  0.14  0.09  0.15  0.08
 0.08  0.17  0.14  0.11  0.04  0.08  0.14  0.06  0.18

In [21]:
save_object("scenario_grid.jld2", res)

In [23]:
grid = load_object("scenario_grid.jld2")

Lattice([3.9677712225053408 4.16037020371943 … 4.16372659274145 4.334347452237141; 4.031548776342026 3.94038115465565 … 4.118351094526266 4.152481274854603; … ; 4.193312761680589 4.201737098645205 … 4.099247950548817 4.050641148485665; 3.8618455995215575 4.214718728122945 … 3.9602295733772532 4.020421172044646;;; 3.9680391768668932 4.028895168282186 … 3.9012427516249946 3.98949119069828; 4.146568671739699 3.968563723668759 … 3.931966989284409 3.8988848032794543; … ; 3.8919873544774855 4.0950358500472674 … 4.057916016128649 3.930322287849103; 4.047292513828673 3.8905646653717985 … 3.9355466328621453 4.018739152989814;;; 4.007822383717923 4.0044176257783874 … 4.012268551028478 4.019089567679289; 4.036137207513852 3.9974773488046003 … 4.015299513734185 3.9879095080358415; … ; 3.9672447305672325 4.026089939369318 … 3.9873445864987165 3.99581885190879; 3.9913236237948433 3.9686028243310454 … 3.964024528275138 4.019167791690985], [[1.0;;], [0.11909 0.15429 … 0.12376 0.10997], [0.096901503064

In [26]:
round.(grid.states[5,:,:],digits=4)

9×3 Matrix{Float64}:
 3.8703  4.1223  4.0231
 4.1201  4.0083  3.9887
 4.0656  4.1002  4.0074
 4.0303  3.9704  4.0171
 4.0902  4.1193  4.0398
 4.2638  3.9036  3.9936
 4.0163  3.8989  3.9776
 4.2642  4.0198  4.0205
 3.8749  4.0194  3.9938

In [41]:
mean(res.states[6,:,3],FrequencyWeights(res.probs[6][1,:]))

4.004744507916889

In [33]:
using  Test
@test grid.states[5,:,:] ≈ res.states[5,:,:]

Test Passed